In [29]:
import os
os.chdir("../")

In [30]:
import glob
import pandas as pd
import time

In [32]:
resultDir = 'D:\Program Files\learn2brancher\\results'
problem = 'setcover' # choices=['setcover', 'cauctions', 'facilities', 'indset']
# sampling_Strategies = ['uniform5','depthK','depthK2'] # choices: uniform5, depthK, depthK2, depthK3
policy = ['GraphConv','Lodi']
seeds = [0,1,2,3,4]
eval_files = glob.glob(f'{resultDir}\{problem}_*.csv')
eval_file = eval_files[-1]

In [33]:
df = pd.read_csv(eval_file)
df = pd.concat([df[df['type']=='small'], df[df['type']=='medium']])
df = df.astype({'nlps': float, 'nnodes' : float})

df = df[df['policy'] != 'internal:relpscost']

# 计算 one-shifted geometric means（未完成）
use uniform5 as basis, devide each instance's result by the corresponding one solved by uniform5. Then compute one-shifted geometric mean grouped by ['type','sampling_strategy']

In [34]:
# process_time = time.process_time()
# perf_time = time.perf_counter()
df2 = df[df['policy'] == 'gcnn:Lodi']          
for i, row in df.iterrows():
    ref_row = df2[
        # (df2['policy'] == df.loc[i].policy) &
        (df2['seed'] == df.loc[i].seed) &
        (df2['type'] == df.loc[i].type) &
        (df2['instance'] == df.loc[i].instance)
    ]
    df.at[i,'stime'] /= ref_row.stime
    df.at[i,'nnodes'] = df.at[i,'nnodes'].astype(float) / ref_row.nnodes.astype(float)
    df.at[i,'nlps'] = df.at[i,'nlps'].astype(float) / ref_row.nlps.astype(float)
    df.at[i,'walltime'] = df.at[i,'walltime'].astype(float) / ref_row.walltime.astype(float)
    df.at[i,'proctime'] = df.at[i,'proctime'].astype(float) / ref_row.proctime.astype(float)
 

In [35]:
import numpy as np

In [37]:
df.loc[(df['policy'] == 'gcnn:GraphConv') & (df['type']=='small'),'stime'].mean()

0.976877795540645

In [38]:
df.groupby(['type','policy']).mean()

seed    nnodes      nlps     stime  gap  ndomchgs  \
type   policy                                                              
medium gcnn:GraphConv     2  0.941917  0.945069  0.926573  0.0         0   
       gcnn:Lodi          2  1.000000  1.000000  1.000000  0.0         0   
small  gcnn:GraphConv     2  0.985406  0.974459  0.976878  0.0         0   
       gcnn:Lodi          2  1.000000  1.000000  1.000000  0.0         0   

                       ncutoffs  walltime  proctime  
type   policy                                        
medium gcnn:GraphConv         0  0.928325  0.927837  
       gcnn:Lodi              0  1.000000  1.000000  
small  gcnn:GraphConv         0  0.981277  0.979159  
       gcnn:Lodi              0  1.000000  1.000000

In [41]:
from scipy import stats

In [42]:
stats.mstats.gmean(df.loc[(df['policy'] == 'gcnn:GraphConv') & (df['type']=='small'),'stime'] + 1) - 1

0.9755389476097125

# 计算 standard deviations
Average per-instance standard deviation (percentage) w.r.t different model training seeds

In [43]:
std_df = df.groupby(['type','policy','instance']).std() / df.groupby(['type','policy','instance']).mean()

In [45]:
std_df.groupby(['type','policy']).mean()

seed    nnodes      nlps    stime  gap  ndomchgs  \
type   policy                                                                 
medium gcnn:GraphConv  0.790569  0.103512  0.105505  0.09365  NaN       NaN   
       gcnn:Lodi       0.790569  0.000000  0.000000  0.00000  NaN       NaN   
small  gcnn:GraphConv  0.790569  0.105612  0.084389  0.05641  NaN       NaN   
       gcnn:Lodi       0.790569  0.000000  0.000000  0.00000  NaN       NaN   

                       ncutoffs  walltime  proctime  
type   policy                                        
medium gcnn:GraphConv       NaN  0.094222  0.093202  
       gcnn:Lodi            NaN  0.000000  0.000000  
small  gcnn:GraphConv       NaN  0.056028  0.053366  
       gcnn:Lodi            NaN  0.000000  0.000000

# 计算 Wins

In [46]:
df_GraphConv = df[df['policy']=='gcnn:GraphConv']
df_Lodi = df[df['policy']=='gcnn:Lodi']
# df_depthK2 = df[df['sampling_strategy']=='depthK2']
df['Wins'] = 0

In [47]:
for i in range(0,len(df_Lodi)):
    Lodi_row = df_Lodi.iloc[i]
    GraphConv_row = df[
        # (df['sampling_strategy'] == 'depthK') &
        (df['policy'] == 'gcnn:GraphConv') &
        (df['seed'] == Lodi_row.seed) &
        (df['type'] == Lodi_row.type) &
        (df['instance'] == Lodi_row.instance)
    ]
    # depthK2_row = df.loc[
    #     (df['sampling_strategy'] == 'depthK2') &
    #     (df['policy'] == uniform5_row.policy) &
    #     (df['seed'] == uniform5_row.seed) &
    #     (df['type'] == uniform5_row.type) &
    #     (df['instance'] == uniform5_row.instance)
    # ]
    winner = np.argmin([Lodi_row.stime, GraphConv_row.stime.iloc[0]])
    winner = ['gcnn:Lodi', 'gcnn:GraphConv'][winner]
    df.loc[ 
        # (df['sampling_strategy'] == winner) &
            (df['policy'] == winner) &
            (df['seed'] == Lodi_row.seed) &
            (df['type'] == Lodi_row.type) &
            (df['instance'] == Lodi_row.instance), 'Wins'] += 1

In [50]:
df.groupby(['type', 'policy']).sum()

seed      nnodes        nlps       stime  gap  \
type   policy                                                          
medium gcnn:GraphConv   200   94.191673   94.506948   92.657299  0.0   
       gcnn:Lodi        200  100.000000  100.000000  100.000000  0.0   
small  gcnn:GraphConv   200   98.540629   97.445927   97.687780  0.0   
       gcnn:Lodi        200  100.000000  100.000000  100.000000  0.0   

                       ndomchgs  ncutoffs    walltime    proctime  Wins  
type   policy                                                            
medium gcnn:GraphConv         0         0   92.832516   92.783733    82  
       gcnn:Lodi              0         0  100.000000  100.000000    18  
small  gcnn:GraphConv         0         0   98.127740   97.915934    59  
       gcnn:Lodi              0         0  100.000000  100.000000    41

In [51]:
df.groupby(['type', 'policy']).std()

seed    nnodes      nlps     stime  gap  ndomchgs  \
type   policy                                                                  
medium gcnn:GraphConv  1.421338  0.114808  0.114896  0.104819  0.0       0.0   
       gcnn:Lodi       1.421338  0.000000  0.000000  0.000000  0.0       0.0   
small  gcnn:GraphConv  1.421338  0.126221  0.096683  0.072707  0.0       0.0   
       gcnn:Lodi       1.421338  0.000000  0.000000  0.000000  0.0       0.0   

                       ncutoffs  walltime  proctime      Wins  
type   policy                                                  
medium gcnn:GraphConv       0.0  0.105868  0.104834  0.386123  
       gcnn:Lodi            0.0  0.000000  0.000000  0.386123  
small  gcnn:GraphConv       0.0  0.077859  0.070738  0.494311  
       gcnn:Lodi            0.0  0.000000  0.000000  0.494311